# RANDOM BOOSTED FORESTS

In [1]:
#import libraries
from http.client import ImproperConnectionState
import numpy as np
import pandas as pd
import xgboost as xgb
import sklearn 
from sklearn.model_selection import train_test_split
import time
#import graphviz
import os
import pickle

/home/luise/anaconda3/envs/ML/lib/python3.10/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
#load data
version_data = "003"
path_data = f'../../data/preprocessed/{version_data}/' 
(x_test, x_train, y_test, y_train, x_predict) = [np.load(f'{path_data}{f}.npy', allow_pickle=True) 
                                                for f in ('X_test', 'X_train', 'y_test', 'y_train',  'X_predict')]

print(f'x_train.shape: {x_train.shape}')
print(f'x_test.shape: {x_test.shape}')
print(f'y_train.shape: {y_train.shape}')
print(f'y_test.shape: {y_test.shape}')
print(f'x_predict.shape: {x_predict.shape}')

x_train.shape: (300587, 100)
x_test.shape: (75147, 100)
y_train.shape: (300587,)
y_test.shape: (75147,)
x_predict.shape: (31354, 100)


### XGBoost
--> random boosted forests

Train model

In [3]:
#train model
xgbo = xgb.XGBRegressor(objective ='reg:squarederror', eval_metric='rmse', n_estimators=300, max_depth=30, learning_rate=0.1, random_state=42)
#
start_time = time.time()  #track the model development time
xgbo.fit(x_train,y_train)
end_time = time.time()
duration = end_time - start_time

print("Training duration: %s sec" % (duration)) 

#predict
y_predict = xgbo.predict(x_test)
RMSE = sklearn.metrics.mean_squared_error(y_test, y_predict, squared=False)
print('RMSE: ', RMSE)

Training duration: 528.0271301269531 sec
RMSE:  0.5351610305741148


SAVE MODEL

In [4]:
model_id = 'xgboost__'+ version_data + '__03'
print(f"Saving model {model_id} ...")

#save model
if not os.path.exists(f'../../models/{model_id}'):
        os.mkdir(f'../../models/{model_id}')
pickle.dump(xgbo, open(f'../../models/{model_id}/model.pkl', 'wb'))

#save results
timestamp = time.strftime("%Y_%m_%d at %H_%M")
print(timestamp)
print(f'RMSE: {RMSE}')

overview = pd.read_csv('../../models/prediction_overview.csv')
new_row = {'model_id': model_id, 'timestamp': timestamp, 'error': RMSE}
overview = pd.concat([overview, pd.DataFrame(new_row, index=[0])])
overview = overview[{'model_id', 'timestamp', 'error'}]
overview.drop_duplicates(inplace=True)
print(overview)
overview.to_csv('../../models/prediction_overview.csv')

Saving model xgboost__003__03 ...
2023_02_25 at 12_35
RMSE: 0.5351610305741148
              model_id            timestamp          error
0   xgboost__first__01  2023_01_06 at 15_52            NaN
1   xgboost__first__02  2023_01_30 at 13_10            NaN
2   xgboost__first__02  2023_01_30 at 13_11       0.259323
3        TF__first__01  2023_01_30 at 14_17  724579.241582
4        TF__first__01  2023_01_30 at 14_27       9.875672
5     xgboost__001__02  2023_01_31 at 17_05       0.292051
6     xgboost__001__02  2023_01_31 at 17_08       0.744116
7          TF__001__02  2023_01_31 at 17_45       5.278455
8          TF__001__02  2023_01_31 at 17_46       5.278455
9     xgboost__001__03  2023_01_31 at 17_57       0.339240
10    xgboost__001__03  2023_01_31 at 18_02       0.338606
11    xgboost__002__03  2023_02_24 at 18_06       1.016552
0     xgboost__003__03  2023_02_25 at 12_35       0.535161


/tmp/ipykernel_12691/4240199680.py:17: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  overview = overview[{'model_id', 'timestamp', 'error'}]


In [9]:
############# use to create new version and DELETE OLD VERSIONS #############

#overview = pd.DataFrame(columns=['model_id', 'timestamp', 'error'])
#overview.to_csv('../models/prediction_overview.csv')